In [2]:
import fastText
import math
import linecache
import numpy as np 
from numpy import random
from random import sample
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import *
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

Using TensorFlow backend.
/home1/zishan/anaconda3/envs/raghav_btp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
set_session(tf.Session(config=config))

In [4]:
ft = fastText.load_model("/home1/zishan/raghav/wiki.hi.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [5]:
def twitter_tokenizer(textline):
    textLine = re.sub(r'http\S+', 'URL', textline)
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    # print(words)
    return(words)

In [6]:
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims, tokenize=True):
    if not tokenize:
        words = textline.split()
    else:
        words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft

In [7]:
def sequential_generator(filename, 
                         batch_size, 
                         labels2Idx:'dict to make output labels',
                         check:'to check if all lines in file are of same length.To check enter the len of line after splitting it by tabs' = None,
                         tokenize:'specify if using twitter tokenzor to preprocess lines'=False, 
                        ):    
    
    f = open(filename)
    n_labels = len(labels2Idx)
    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
        batch_labels = np.zeros((batch_size, len(labels2Idx)))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
            data = line.strip().split('\t')
            if check:
                if len(data)!=check:
                    i-=1
                    continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims, tokenize= tokenize)
            if len(labels2Idx)==2:
                batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            else:
                batch_labels[i] = to_categorical(labels2Idx[data[1]], n_labels)
        yield ([batch_features_ft], batch_labels)

In [8]:
def train_dev_sentences(filetrain, filedev, check:'to check if lines of file are all same lenght after separating by tab'):
    labels2Idx = {}
    train_lines = [line.strip().split("\t") for line in open(filetrain) if len(line.split('\t'))==check]
    dev_lines = [line.strip().split("\t") for line in open(filedev) if len(line.strip().split('\t'))==check]
    train_sentences = [x[0] for x in train_lines]
    for dataset in [train_lines, dev_lines]:
        for line in dataset:
            label = line[1]
            if label not in labels2Idx.keys():
                labels2Idx[label]= len(labels2Idx)
                
    train_labels = [labels2Idx[x[1]] for x in train_lines]
    dev_sentences = [x[0] for x in dev_lines]
    dev_labels = [labels2Idx[x[1]] for x in dev_lines]
    return (train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx)

In [9]:
train_file = '/home1/zishan/raghav/Data/train.txt'
dev_file = '/home1/zishan/raghav/Data/dev.txt'
train_sentences, train_labels, dev_sentences, dev_labels, labels2Idx = train_dev_sentences(train_file, dev_file, 2)

In [10]:
n_words = 0
for sentence in train_sentences:
    n_words+=len(sentence)
print(n_words)

112581


In [11]:
from collections import Counter
print(Counter(train_labels))
print(labels2Idx)

Counter({0: 382, 1: 193, 2: 153, 6: 147, 5: 123, 3: 76, 4: 64, 7: 45, 8: 21})
{'SADNESS': 0, 'SYMPATHY/PENSIVENESS': 1, 'OPTIMISM': 2, 'JOY': 3, 'DISGUST': 4, 'FEAR/ANXIETY': 5, 'NO-EMOTION': 6, 'ANGER': 7, 'SURPRISE': 8}


In [27]:
print(train_sentences[:10])
print(labels2Idx)
print(len(train_labels))

[': पश्चिम बंगाल के मिदनापुर और सुंदरवन के तटीय इलाकों में रविवार को आए तूफानी चक्रवात की वजह से हजारों लोग बेघर हो गए हैं।', 'ये मछुआरे 60 नावें लेकर ताइवान की सीमा में आए।', 'उनके जरिए सूखा प्रभावित गांवों और कस्बों में पानी की आपूर्ति की जा रही है।', 'लोगों तक दवा व खाद्य सामग्री पहुंचाई जा रही है।', 'नैनीताल की वरिष्ठ पुलिस अधीक्षक स्वीटी अग्रवाल ने बताया कि नैनीताल के जंगलों में आग बुझाने की कार्रवाई प्रभावी ढंग से की जा रही है।', 'वरिष्ठ पुलिस अधीक्षक (कुपवाड़ा) एजाज अहमद ने कहा कि तीनों आतंकवादी शिविर में घुस गए थे।', 'हालांकि अमेरिका ने किसी भी देश से मदद की गुहार नहीं लगाई थी लेकिन दर्जनों देश खुद ही राहत सामग्री और धन के जरिए मदद करने को तत्पर हैं। अमेरिकी विदेश विभाग ने कहा कि अब तक 40 से अधिक देशों और अंतरराष्ट्रीय संगठनों ने उसे मदद देने की पेशकश की है और इस संख्या में लगातार बढ़ोतरी होती जा रही है।', 'राज्य के चंदवली इलाके में २५.८ मिलीमीटर बारिश रिकॅर्ड की गयी जबकि बालासोर में १२.७ मिलीमीटर, भुवनेशवर ६.२ मिलीमीटर, पुरी में ०.२ मिलीमीटर और गोपालपुर में १९.५ मिलीमीटर बारिश 

In [26]:
n_labels = len(labels2Idx)

In [106]:
def compile_model_bilstm(no_labels:'total labels for classification'):
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)
    model_concatenated = Flatten()(lstm_block)
    model_concatenated = Dense(100)(model_concatenated)
    model_output = Dense(no_labels, activation = "softmax")(model_concatenated)
    new_model = Model(model_input_embedding, model_output)
    new_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
    new_model.summary()
    return new_model

In [17]:
def compile_model_bilstm_cnn(no_labels:'total labels for classification'):
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)

    filter_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in filter_sizes:
        conv = Conv1D(
            filters = 200,
            kernel_size = sz,
            padding = 'valid',
            strides = 1
        )(lstm_block)
        conv = LeakyReLU()(conv)
        conv = GlobalMaxPooling1D()(conv)
        conv = Dropout(0.5)(conv)
        conv_blocks.append(conv)
    model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    model_concatenated = Dense(100)(model_concatenated)
    model_concatenated = LeakyReLU()(model_concatenated)
    model_output = Dense(no_labels, activation = "softmax")(model_concatenated)
    new_model = Model(model_input_embedding, model_output)
    new_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
    new_model.summary()
    return new_model

In [162]:
model = compile_model_bilstm_cnn(no_labels = n_labels)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_28 (InputLayer)           (None, 75, 300)      0                                            
__________________________________________________________________________________________________
bidirectional_28 (Bidirectional (None, 75, 200)      320800      input_28[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_112 (LeakyReLU)     (None, 75, 200)      0           bidirectional_28[0][0]           
__________________________________________________________________________________________________
conv1d_64 (Conv1D)              (None, 73, 200)      120200      leaky_re_lu_112[0][0]            
__________________________________________________________________________________________________
conv1d_65 

In [163]:
train_file = '/home1/zishan/raghav/Data/train.txt'
weights_file ='/home1/zishan/raghav/weights/bilstm_3cnn_dropout=0.5.h5'
log_file = '/home1/zishan/raghav/logs/bilstm_3cnn_dropout=0.5.txt'
batch_size = 16
check_for_generator = 2
labels2Idx = labels2Idx
tokenize = True
samples_per_epoch = len(train_sentences)
steps_per_epoch = math.ceil(samples_per_epoch / batch_size)

In [164]:
max_f1 = 0
for epoch in range(200):
    print("Epoch {}".format(epoch))
    model.fit_generator(sequential_generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                                             labels2Idx= labels2Idx,tokenize= tokenize),
                        steps_per_epoch= steps_per_epoch, epochs=1,)

    testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
    for i in range(len(dev_sentences)):
        testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
    results = model.predict(testset_features)


    predLabels = results.argmax(axis=-1)
    devLabels = dev_labels
    f1 = f1_score(devLabels, predLabels, average='macro') # offensive is the major class. So other is minor
    r = recall_score(devLabels, predLabels, average='macro')
    p = precision_score(devLabels, predLabels, average='macro')
    a = accuracy_score(devLabels, predLabels)
    
    if f1> max_f1:
        model.save_weights(weights_file)
        with open(log_file,'a+') as f:
            text = str(epoch)+', a: '+str(a) +', f1:' +str(f1) +'\n'
            f.write(text)
        max_f1 = f1

    print(a,f1)

Epoch 0
Epoch 1/1
76/76 [==============================] - 37s 484ms/step - loss: 1.9961 - acc: 0.2969


/home1/zishan/anaconda3/envs/raghav_btp/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home1/zishan/anaconda3/envs/raghav_btp/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.37 0.06001622060016221
Epoch 1
Epoch 1/1
76/76 [==============================] - 21s 283ms/step - loss: 1.9416 - acc: 0.3199
0.37 0.06001622060016221
Epoch 2
Epoch 1/1
76/76 [==============================] - 21s 278ms/step - loss: 1.9041 - acc: 0.3298
0.37666666666666665 0.08070807080708071
Epoch 3
Epoch 1/1
76/76 [==============================] - 22s 286ms/step - loss: 1.8352 - acc: 0.3495
0.38333333333333336 0.11185098141619881
Epoch 4
Epoch 1/1
76/76 [==============================] - 21s 281ms/step - loss: 1.7664 - acc: 0.3602
0.41333333333333333 0.16583755329957361
Epoch 5
Epoch 1/1
76/76 [==============================] - 21s 278ms/step - loss: 1.6694 - acc: 0.3914
0.41 0.16689465072214396
Epoch 6
Epoch 1/1
76/76 [==============================] - 21s 281ms/step - loss: 1.5909 - acc: 0.4243
0.39666666666666667 0.16753387089194793
Epoch 7
Epoch 1/1
76/76 [==============================] - 22s 286ms/step - loss: 1.5064 - acc: 0.4342
0.4166666666666667 0.2093236733346105
Epoch 

76/76 [==============================] - 27s 349ms/step - loss: 0.0732 - acc: 0.9770
0.4033333333333333 0.28637521588317
Epoch 119
Epoch 1/1
76/76 [==============================] - 27s 357ms/step - loss: 0.0688 - acc: 0.9844
0.4266666666666667 0.2776516923415956
Epoch 120
Epoch 1/1
76/76 [==============================] - 27s 360ms/step - loss: 0.0591 - acc: 0.9770
0.4033333333333333 0.2657445523593661
Epoch 121
Epoch 1/1
76/76 [==============================] - 27s 361ms/step - loss: 0.0732 - acc: 0.9737
0.41333333333333333 0.27683036751073103
Epoch 122
Epoch 1/1
76/76 [==============================] - 27s 359ms/step - loss: 0.0645 - acc: 0.9794
0.41 0.25436453897508476
Epoch 123
Epoch 1/1
76/76 [==============================] - 27s 351ms/step - loss: 0.0605 - acc: 0.9770
0.4166666666666667 0.27541189460694104
Epoch 124
Epoch 1/1
76/76 [==============================] - 27s 354ms/step - loss: 0.0600 - acc: 0.9786
0.43333333333333335 0.3055407652467522
Epoch 125
Epoch 1/1
76/76 [===

# transfer learning

In [12]:
word_vectors_ft = {}
def process_features_crosslingual(textline, nb_sequence_length, nb_embedding_dims, tokenize=True, transmat = None):
    if not tokenize:
        words = textline.split()
    else:
        words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            wv = np.matmul(wv, transmat) # applying transformation on the word vector to make the vector in same space
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft

In [13]:
def sequential_generator_crosslingual(filename, 
                         batch_size, 
                         labels2Idx:'dict to make output labels',
                         transmat:'Matrix to make embeddings in same vector space'= None,
                         check:'to check if all lines in file are of same length.To check enter the len of line after splitting it by tabs' = None,
                         tokenize:'specify if using twitter tokenzor to preprocess lines'=False,  
                        ):    
    
    f = open(filename)
    n_labels = len(labels2Idx)
    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
        batch_labels = np.zeros((batch_size, len(labels2Idx)))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
            data = line.strip().split('\t')
            if check:
                if len(data)!=check:
                    i-=1
                    continue
            batch_features_ft[i] = process_features_crosslingual(data[0], nb_sequence_length, nb_embedding_dims, tokenize= tokenize, transmat = transmat)
            if len(labels2Idx)==2:
                batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            else:
                batch_labels[i] = to_categorical(labels2Idx[data[1]], n_labels)
#         print(batch_features_ft.shape, batch_labels)
        yield ([batch_features_ft], batch_labels)

In [30]:
def test_model_tl_unfreezing(generator, 
               train_sentences, 
               devLabels, 
               number_of_tests,
               number_of_epochs,
               filename_to_log, 
               labels2Idx,
               filename_to_save_weigths,
               batch_size, 
               unfreezing_strategy: 'list containing a tuple of indices to unfreeze at each step',
               train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', 
               pos_label:'only if binary f1',
               load_model_weights=False,
               model_weights_file:'give filepath as str'=None, 
               tokenize=True,
               nb_sequence_length = nb_sequence_length,
               nb_embedding_dims= nb_embedding_dims, 
               transmat: 'matrix if crosslingual training'=None,
               check_for_generator=None):
    
    f = open(filename_to_log, 'w', encoding='utf-8')
    f.close()
   
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model_bilstm_cnn(13)

        # transfer learning
        if load_model_weights and model_weights_file:
                model.load_weights(model_weights_file)
                print("removing top layer")
                model.layers.pop()
                output = Dense(len(labels2Idx), activation = 'softmax')(model.layers[-1].output)
                final_model = Model(inputs=model.input, outputs=[output])

        samples_per_epoch = len(train_sentences)
        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)

        max_f1=0
        max_p=0
        max_r=0
        max_a=0
        
        # load pretrained weights
        # model.compile
        # save tmp weights
        # iterate over layers
        #    load tmp weights
        #    iterate over epochs
        #        unfreeze top frozen layer
        #        save best model as tmp weights
        
        
        final_model.save(filename_to_save_weigths)
        
        # layers_to_unfreeze = [18, 16, 3, 1]
        
        for ulayer in unfreezing_strategy:
            print("unfreezing " + final_model.layers[ulayer[0]].name)
            print("---------------------------------------")
            final_model.load_weights(filename_to_save_weigths)            
            for i, layer in enumerate(final_model.layers):
                
                # TF strategy: gradual unfreezing
                #if i >= ulayer:
                #    layer.trainable = True
                #else:
                #    layer.trainable = False
                # 
                ## TF strategy: single
                
                if i >= ulayer[1] and i <= ulayer[0]:
                    layer.trainable = True
                else:
                    layer.trainable = False
                    
                print(str(i) + ' ' + layer.name + ' ' + str(layer.trainable))
            final_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
        
            for epoch in range(epochs):
                print("Epoch: %d/%d" %(epoch+1, epochs))
                final_model.fit_generator(
                    generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                              labels2Idx= labels2Idx, transmat = transmat, tokenize= tokenize), 
                    steps_per_epoch= steps_per_epoch, epochs=1
                )

                testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
                for i in range(len(dev_sentences)):
                    testset_features[i] = process_features_crosslingual(dev_sentences[i], nb_sequence_length, nb_embedding_dims, transmat= transmat)
                results = final_model.predict(testset_features)

                predLabels = results.argmax(axis=-1)
                devLabels = devLabels
                f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
                r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
                p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
                a = accuracy_score(devLabels, predLabels)
                if max_f1 < f1:
                    print("model saved. F1 is %f" %(f1))
                    final_model.save_weights(filename_to_save_weigths)
                    max_f1 = f1
                    max_p = p
                    max_r = r
                    max_a = a
                text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
                print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
        to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
        print(to_write)
        with open(filename_to_log,'a') as f:
            f.write(to_write)
        total_f1+=max_f1
        total_prec+=max_p
        total_acc+=max_a
        total_recall+=max_r    
        print("*****************************************************************************")
    final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
    print(final_text)
    with open(filename_to_log,'a') as f:
        f.write(final_text)

In [31]:
# list of tuples. Every tuple contains range of layers which need to be unfrozen. Rest all are frozen
single_unfreeze_bottom_up = [(18, 18), (17, 16), (15, 3), (2, 1), (18,1)] 
single_unfreeze_top_down = [(18, 18),   (2, 1),(15, 3), (17, 16), (18,1)]
all_unfreeze = [(18,1)]
gradual_unfreezing = [(18,18), (18,16), (18,3), (18,1)]

strings =['suf_bu', 'suf_td','all_unfreeze','gradual_unfreeze']
# strings = ['suf_td','all_unfreeze', 'gradual_unfreeze']
# strings=['gradual_unfreeze']
unfreeze_strategy = [single_unfreeze_bottom_up, single_unfreeze_top_down, all_unfreeze, gradual_unfreezing]
# unfreeze_strategy = [gradual_unfreezing]
# unfreeze_strategy = [ single_unfreeze_top_down, all_unfreeze, gradual_unfreezing]


In [29]:
for i in range(len(strings)):
    string = strings[i]
    print("approach: %s" %(string))
    
    generator = sequential_generator_crosslingual
    train_sentences = train_sentences
    devLabels = dev_labels
    number_of_tests = 1
    transmat = np.loadtxt('/home1/zishan/raghav/fastText_multilingual/alignment_matrices/hi.txt')
    number_of_epochs = 200
    labels2Id = labels2Idx
    log_file = '/home1/zishan/raghav/logs/tl_crowdflower_' +string+'.txt' 
    print("log file: %s" %(log_file))
    weights_file='/home1/zishan/raghav/weights/tl_crowdflower_'+string+'.h5'
    print("save weights file: %s" %(weights_file))
    batch_size=16
    train_file='/home1/zishan/raghav/Data/train.txt'
    f1_measure='macro'
    pos_label=1
    strategy = unfreeze_strategy[i]
    print(strategy)
    load_model_weights=True
    load_weights_file = '/home1/zishan/raghav/weights/pretrain_crowdflower_bilstm_3cnn.h5'
    nb_sequence_length = nb_sequence_length
    nb_embedding_dims= nb_embedding_dims
    check_for_generator=2
    
    test_model_tl_unfreezing(generator=generator, 
           train_sentences=train_sentences, 
           devLabels=devLabels, 
           number_of_tests= number_of_tests,
           number_of_epochs=number_of_epochs, 
           filename_to_log=log_file, 
           labels2Idx = labels2Id,
           filename_to_save_weigths=weights_file,
           batch_size=batch_size,
           unfreezing_strategy = strategy,       
           train_file=train_file, 
           f1_measure=f1_measure, 
           pos_label=pos_label, 
           load_model_weights=True,
           model_weights_file = load_weights_file, 
           nb_sequence_length=nb_sequence_length, 
           nb_embedding_dims=nb_embedding_dims, 
           transmat = transmat,
           check_for_generator= check_for_generator)

approach: suf_bu
log file: /home1/zishan/raghav/logs/tl_crowdflower_suf_bu.txt
save weights file: /home1/zishan/raghav/weights/tl_crowdflower_suf_bu.h5
[(18, 18), (17, 16), (15, 3), (2, 1), (18, 1)]
Test 1/1
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, 75, 300)      0                                            
__________________________________________________________________________________________________
bidirectional_14 (Bidirectional (None, 75, 200)      320800      input_14[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_66 (LeakyReLU)      (None, 75, 200)      0           bidirectional_14[0][0]           
__________________________________________________________________________________________________


/home1/zishan/anaconda3/envs/raghav_btp/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home1/zishan/anaconda3/envs/raghav_btp/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


model saved. F1 is 0.091914
Test-Data: Prec: 0.088, Rec: 0.117, F1: 0.092, Acc: 0.310
Epoch: 2/200
Epoch 1/1
76/76 [==============================] - 6s 82ms/step - loss: 2.2449 - acc: 0.2401
model saved. F1 is 0.097999
Test-Data: Prec: 0.141, Rec: 0.131, F1: 0.098, Acc: 0.350
Epoch: 3/200
Epoch 1/1
76/76 [==============================] - 6s 82ms/step - loss: 2.1268 - acc: 0.2434
Test-Data: Prec: 0.116, Rec: 0.132, F1: 0.095, Acc: 0.370
Epoch: 4/200
Epoch 1/1
76/76 [==============================] - 6s 83ms/step - loss: 2.0965 - acc: 0.2845
Test-Data: Prec: 0.071, Rec: 0.123, F1: 0.081, Acc: 0.367
Epoch: 5/200
Epoch 1/1
76/76 [==============================] - 6s 83ms/step - loss: 2.0861 - acc: 0.2697
Test-Data: Prec: 0.083, Rec: 0.132, F1: 0.092, Acc: 0.373
Epoch: 6/200
Epoch 1/1
76/76 [==============================] - 6s 83ms/step - loss: 2.0685 - acc: 0.2780
Test-Data: Prec: 0.084, Rec: 0.128, F1: 0.088, Acc: 0.373
Epoch: 7/200
Epoch 1/1
76/76 [==============================] - 6s

76/76 [==============================] - 6s 84ms/step - loss: 1.9608 - acc: 0.3100
Test-Data: Prec: 0.085, Rec: 0.129, F1: 0.089, Acc: 0.377
Epoch: 52/200
Epoch 1/1
76/76 [==============================] - 6s 84ms/step - loss: 1.9324 - acc: 0.3133
Test-Data: Prec: 0.093, Rec: 0.131, F1: 0.091, Acc: 0.383
Epoch: 53/200
Epoch 1/1
76/76 [==============================] - 6s 83ms/step - loss: 1.9379 - acc: 0.3248
Test-Data: Prec: 0.089, Rec: 0.123, F1: 0.081, Acc: 0.377
Epoch: 54/200
Epoch 1/1
76/76 [==============================] - 6s 84ms/step - loss: 1.9661 - acc: 0.3133
Test-Data: Prec: 0.085, Rec: 0.129, F1: 0.089, Acc: 0.377
Epoch: 55/200
Epoch 1/1
76/76 [==============================] - 7s 92ms/step - loss: 1.9464 - acc: 0.3232
Test-Data: Prec: 0.079, Rec: 0.122, F1: 0.079, Acc: 0.373
Epoch: 56/200
Epoch 1/1
76/76 [==============================] - 7s 86ms/step - loss: 1.9436 - acc: 0.3166
Test-Data: Prec: 0.108, Rec: 0.123, F1: 0.082, Acc: 0.377
Epoch: 57/200
Epoch 1/1
76/76 [===

76/76 [==============================] - 6s 84ms/step - loss: 1.9388 - acc: 0.3158
Test-Data: Prec: 0.079, Rec: 0.123, F1: 0.080, Acc: 0.377
Epoch: 102/200
Epoch 1/1
76/76 [==============================] - 6s 84ms/step - loss: 1.9437 - acc: 0.3215
Test-Data: Prec: 0.097, Rec: 0.123, F1: 0.081, Acc: 0.377
Epoch: 103/200
Epoch 1/1
76/76 [==============================] - 6s 85ms/step - loss: 1.9396 - acc: 0.3183
Test-Data: Prec: 0.097, Rec: 0.123, F1: 0.081, Acc: 0.377
Epoch: 104/200
Epoch 1/1
76/76 [==============================] - 6s 82ms/step - loss: 1.9503 - acc: 0.3133
Test-Data: Prec: 0.079, Rec: 0.123, F1: 0.080, Acc: 0.377
Epoch: 105/200
Epoch 1/1
76/76 [==============================] - 6s 85ms/step - loss: 1.9375 - acc: 0.3125
Test-Data: Prec: 0.079, Rec: 0.123, F1: 0.080, Acc: 0.377
Epoch: 106/200
Epoch 1/1
76/76 [==============================] - 6s 82ms/step - loss: 1.9539 - acc: 0.3257
Test-Data: Prec: 0.108, Rec: 0.123, F1: 0.082, Acc: 0.377
Epoch: 107/200
Epoch 1/1
76/7

...

76/76 [==============================] - 20s 257ms/step - loss: 0.1176 - acc: 0.9605
Test-Data: Prec: 0.274, Rec: 0.266, F1: 0.263, Acc: 0.393
Epoch: 94/200
Epoch 1/1
76/76 [==============================] - 20s 266ms/step - loss: 0.0956 - acc: 0.9737
Test-Data: Prec: 0.284, Rec: 0.272, F1: 0.268, Acc: 0.403
Epoch: 95/200
Epoch 1/1
76/76 [==============================] - 20s 262ms/step - loss: 0.0967 - acc: 0.9679
Test-Data: Prec: 0.280, Rec: 0.253, F1: 0.260, Acc: 0.383
Epoch: 96/200
Epoch 1/1
76/76 [==============================] - 20s 260ms/step - loss: 0.1019 - acc: 0.9630
Test-Data: Prec: 0.248, Rec: 0.243, F1: 0.240, Acc: 0.383
Epoch: 97/200
Epoch 1/1
76/76 [==============================] - 20s 260ms/step - loss: 0.0873 - acc: 0.9712
Test-Data: Prec: 0.333, Rec: 0.302, F1: 0.308, Acc: 0.427
Epoch: 98/200
Epoch 1/1
76/76 [==============================] - 20s 260ms/step - loss: 0.0939 - acc: 0.9679
Test-Data: Prec: 0.302, Rec: 0.281, F1: 0.287, Acc: 0.397
Epoch: 99/200
Epoch 1/

76/76 [==============================] - 19s 253ms/step - loss: 0.0633 - acc: 0.9811
Test-Data: Prec: 0.285, Rec: 0.275, F1: 0.273, Acc: 0.397
Epoch: 143/200
Epoch 1/1
76/76 [==============================] - 19s 255ms/step - loss: 0.0506 - acc: 0.9811
Test-Data: Prec: 0.272, Rec: 0.258, F1: 0.260, Acc: 0.380
Epoch: 144/200
Epoch 1/1
76/76 [==============================] - 19s 254ms/step - loss: 0.1004 - acc: 0.9638
Test-Data: Prec: 0.282, Rec: 0.259, F1: 0.264, Acc: 0.390
Epoch: 145/200
Epoch 1/1
76/76 [==============================] - 19s 249ms/step - loss: 0.0645 - acc: 0.9786
Test-Data: Prec: 0.277, Rec: 0.252, F1: 0.261, Acc: 0.370
Epoch: 146/200
Epoch 1/1
76/76 [==============================] - 19s 255ms/step - loss: 0.0640 - acc: 0.9811
Test-Data: Prec: 0.278, Rec: 0.243, F1: 0.251, Acc: 0.363
Epoch: 147/200
Epoch 1/1
76/76 [==============================] - 19s 250ms/step - loss: 0.0704 - acc: 0.9778
Test-Data: Prec: 0.314, Rec: 0.274, F1: 0.283, Acc: 0.390
Epoch: 148/200
Ep

76/76 [==============================] - 19s 247ms/step - loss: 0.0593 - acc: 0.9852
Test-Data: Prec: 0.275, Rec: 0.230, F1: 0.236, Acc: 0.397
Epoch: 192/200
Epoch 1/1
76/76 [==============================] - 19s 248ms/step - loss: 0.0706 - acc: 0.9794
Test-Data: Prec: 0.273, Rec: 0.247, F1: 0.254, Acc: 0.370
Epoch: 193/200
Epoch 1/1
76/76 [==============================] - 19s 252ms/step - loss: 0.0653 - acc: 0.9753
model saved. F1 is 0.329118
Test-Data: Prec: 0.417, Rec: 0.307, F1: 0.329, Acc: 0.410
Epoch: 194/200
Epoch 1/1
76/76 [==============================] - 19s 248ms/step - loss: 0.0475 - acc: 0.9885
Test-Data: Prec: 0.274, Rec: 0.266, F1: 0.264, Acc: 0.410
Epoch: 195/200
Epoch 1/1
76/76 [==============================] - 19s 251ms/step - loss: 0.0719 - acc: 0.9762
Test-Data: Prec: 0.260, Rec: 0.256, F1: 0.247, Acc: 0.380
Epoch: 196/200
Epoch 1/1
76/76 [==============================] - 19s 249ms/step - loss: 0.0819 - acc: 0.9786
Test-Data: Prec: 0.262, Rec: 0.264, F1: 0.259, 

76/76 [==============================] - 6s 85ms/step - loss: 2.0485 - acc: 0.2812
Test-Data: Prec: 0.105, Rec: 0.127, F1: 0.087, Acc: 0.380
Epoch: 8/200
Epoch 1/1
76/76 [==============================] - 7s 87ms/step - loss: 1.9930 - acc: 0.2977
Test-Data: Prec: 0.076, Rec: 0.135, F1: 0.092, Acc: 0.373
Epoch: 9/200
Epoch 1/1
76/76 [==============================] - 7s 88ms/step - loss: 1.9928 - acc: 0.2969
Test-Data: Prec: 0.087, Rec: 0.135, F1: 0.094, Acc: 0.383
Epoch: 10/200
Epoch 1/1
75/76 [============================>.] - ETA: 0s - loss: 1.9774 - acc: 0.2975

KeyboardInterrupt: 

In [27]:
model = compile_model_bilstm_cnn(4)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 75, 300)      0                                            
__________________________________________________________________________________________________
bidirectional_13 (Bidirectional (None, 75, 200)      320800      input_13[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_61 (LeakyReLU)      (None, 75, 200)      0           bidirectional_13[0][0]           
__________________________________________________________________________________________________
conv1d_37 (Conv1D)              (None, 73, 200)      120200      leaky_re_lu_61[0][0]             
__________________________________________________________________________________________________
conv1d_38 

In [142]:
model.load_weights('/home1/zishan/raghav/weights/tl_')

In [143]:
model.layers.pop()

In [144]:
output = Dense(n_labels, activation='softmax')(model.layers[-1].output)

In [145]:
final_model = Model(inputs=model.input, outputs=[output])

In [146]:
final_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [147]:
train_file = '/home1/zishan/raghav/Data/train_31_aug.txt'
weights_file ='/home1/zishan/raghav/weights/tl_bilstm_3cnn_dropout=0.8.h5'
log_file = '/home1/zishan/raghav/logs/tl_bilstm_3cnn_dropout=0.8.txt'
batch_size = 16
check_for_generator = 2
labels2Idx = labels2Idx
tokenize = True
samples_per_epoch = len(train_sentences)
steps_per_epoch = math.ceil(samples_per_epoch / batch_size)

In [148]:
max_f1 = 0
for epoch in range(200):
    print("Epoch {}".format(epoch))
    final_model.fit_generator(sequential_generator(filename = train_file, batch_size = batch_size, check = check_for_generator, 
                                             labels2Idx= labels2Idx,tokenize= tokenize),
                        steps_per_epoch= steps_per_epoch, epochs=1,)

    testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
    for i in range(len(dev_sentences)):
        testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
    results = final_model.predict(testset_features)


    #             idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
    predLabels = results.argmax(axis=-1)
    devLabels = dev_labels
    f1 = f1_score(devLabels, predLabels, average='macro') # offensive is the major class. So other is minor
    r = recall_score(devLabels, predLabels, average='macro')
    p = precision_score(devLabels, predLabels, average='macro')
    a = accuracy_score(devLabels, predLabels)
    
    if f1> max_f1:
        final_model.save_weights(weights_file)
        with open(log_file,'a+') as f:
            text = str(epoch)+', a: '+str(a) +', f1:' +str(f1) +'\n'
            f.write(text)
        max_f1 = f1

    print(a,f1)

Epoch 0
Epoch 1/1
76/76 [==============================] - 41s 536ms/step - loss: 2.7022 - acc: 0.1908


/home1/zishan/anaconda3/envs/raghav_btp/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home1/zishan/anaconda3/envs/raghav_btp/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.33 0.09311940659968022
Epoch 1
Epoch 1/1
76/76 [==============================] - 24s 312ms/step - loss: 2.2354 - acc: 0.2344
0.35333333333333333 0.08621111023199796
Epoch 2
Epoch 1/1
76/76 [==============================] - 24s 313ms/step - loss: 2.0822 - acc: 0.2615
0.37333333333333335 0.08090934406723879
Epoch 3
Epoch 1/1
76/76 [==============================] - 24s 319ms/step - loss: 1.9926 - acc: 0.3043
0.38 0.09135211160527616
Epoch 4
Epoch 1/1
76/76 [==============================] - 25s 323ms/step - loss: 1.9912 - acc: 0.2887
0.37333333333333335 0.08629416678952283
Epoch 5
Epoch 1/1
76/76 [==============================] - 24s 315ms/step - loss: 1.9107 - acc: 0.3232
0.37 0.08473625140291807
Epoch 6
Epoch 1/1
76/76 [==============================] - 24s 316ms/step - loss: 1.8900 - acc: 0.3199
0.37 0.08374150265681124
Epoch 7
Epoch 1/1
76/76 [==============================] - 24s 316ms/step - loss: 1.8639 - acc: 0.3215
0.37333333333333335 0.09682471146201715
Epoch 8
Epoch 1/1
7

76/76 [==============================] - 26s 342ms/step - loss: 0.9832 - acc: 0.6505
0.4033333333333333 0.2754653506127467
Epoch 61
Epoch 1/1
76/76 [==============================] - 25s 334ms/step - loss: 0.9621 - acc: 0.6579
0.4166666666666667 0.2725757673147752
Epoch 62
Epoch 1/1
76/76 [==============================] - 25s 335ms/step - loss: 0.9180 - acc: 0.6727
0.4033333333333333 0.2726733175292269
Epoch 63
Epoch 1/1
76/76 [==============================] - 25s 332ms/step - loss: 0.9255 - acc: 0.6743
0.4166666666666667 0.2658217817972259
Epoch 64
Epoch 1/1
76/76 [==============================] - 25s 333ms/step - loss: 0.8980 - acc: 0.6801
0.39666666666666667 0.31572072934606504
Epoch 65
Epoch 1/1
76/76 [==============================] - 25s 323ms/step - loss: 0.9029 - acc: 0.6735
0.4033333333333333 0.26747882036842274
Epoch 66
Epoch 1/1
76/76 [==============================] - 24s 322ms/step - loss: 0.8845 - acc: 0.6785
0.4166666666666667 0.30850515724795646
Epoch 67
Epoch 1/1
76

76/76 [==============================] - 21s 270ms/step - loss: 0.3173 - acc: 0.8865
0.4266666666666667 0.28640305580047937
Epoch 179
Epoch 1/1
76/76 [==============================] - 21s 270ms/step - loss: 0.3262 - acc: 0.8988
0.41333333333333333 0.2860225359389417
Epoch 180
Epoch 1/1
76/76 [==============================] - 21s 271ms/step - loss: 0.3194 - acc: 0.8857
0.4033333333333333 0.2802087340488381
Epoch 181
Epoch 1/1
76/76 [==============================] - 21s 271ms/step - loss: 0.3524 - acc: 0.8791
0.41 0.28040802586545527
Epoch 182
Epoch 1/1
76/76 [==============================] - 20s 269ms/step - loss: 0.2719 - acc: 0.9054
0.38666666666666666 0.2605183038877569
Epoch 183
Epoch 1/1
76/76 [==============================] - 21s 271ms/step - loss: 0.3233 - acc: 0.8799
0.4166666666666667 0.3012017866728012
Epoch 184
Epoch 1/1
76/76 [==============================] - 20s 270ms/step - loss: 0.3242 - acc: 0.8873
0.43 0.311414298078663
Epoch 185
Epoch 1/1
76/76 [=================